# Assignment Set 1
## 1.1 Vibrating string

The problem at hand is the numerical solution of a one-dimensional wave equation, 
which might describe the vibrations of a uniform string, the transport of voltage 
and current along a lossless transmission line, the pressure and flow rate of a 
compressible liquid or gas in a pipe, sound waves in gases or liquids, and optical waves. 
In this example, the domain of the wave equation is a string.

The one-dimensional wave equation is

$$
\frac{\partial^2 \Psi}{\partial t^2}
=
c^2 \frac{\partial^2 \Psi}{\partial x^2}
$$

The solution $\Psi(x,t)$ is the vibration amplitude expressed as a function 
of position and time. The problem becomes fully posed with the addition of 
boundary conditions on the spatial domain, and initial position and velocity 
distributions.

We attempt a numerical solution method by introducing a uniform discretization 
of the spatial and temporal domains, and approximating the partial differential 
equation by finite difference expressions. The grid points of this problem consist 
of discrete nodal points at which $\Psi$ is to be evaluated.

**A. (0.5 point)** Discretize the wave equation, and write it in a form suitable for implementing in a computer program. Assume that the boundaries are fixed, 
$\Psi(x=0,t)=0$, $\Psi(x=L,t)=0$. $L$ is the length of the string. Take $L=1$ for simplicity. Divide the string in $N$ intervals, so that the interval length is 
$\Delta x = L/N$. Also consider the boundary cases.

If you use Euler’s method, you need to use both $\Psi(x,t)$ and $\Psi_t(x,t)$ as variables. Or use the stepping method from the lectures, which uses $\Psi$ at the two most recent time points to calculate it at the next one.

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import json